In [12]:
import albumentations as A
from retriever import TrainRetriever, get_valid_transforms

import numpy as np
import torch
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt

In [13]:
height = 512
width = 512
WH = (width, height)

In [27]:
data_path = Path('../../../Datasets/comma10k/')
rand = 33 #np.random.choice(range(len(image_names)))
target_dir = '../ppt_output/'

os.makedirs(target_dir, exist_ok=True)
image_names = [x for x in os.listdir(data_path/'imgs') if x.endswith('9.png')]
image_name = image_names[rand]
cropper = get_valid_transforms(height, width)
img = cropper(image=cv2.imread(str(data_path/f'imgs/{image_name}')))['image']

In [6]:
def pad_to_multiple(x, k=32):
    return int(k*(np.ceil(x/k)))

In [22]:
transform_names = ['horizontal_flip', 'gaussian_noise', 'grid_distortion', 'elastic', 'shift_scale_rotate', 
                   'optical_distortion', 'clahe', 'random_brightness', 'random_gamma', 'iso_noise', 'sharpen',
                   'blur', 'motion_blur', 'random_contrast', 'hue_saturation', 'resize', 'cutout', 'pad_if_needed']

transforms = [A.HorizontalFlip(p=1.0),
              A.IAAAdditiveGaussianNoise(p=1.0),
              A.GridDistortion(border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, p=1.0),
              A.ElasticTransform(alpha_affine=10, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, p=1.0),
              A.ShiftScaleRotate(shift_limit=0,
                                 scale_limit=0,
                                 rotate_limit=10,
                                 border_mode=cv2.BORDER_CONSTANT,
                                 value=0,
                                 mask_value=0,
                                 p=1.0),
              A.OpticalDistortion(border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, p=1.0),
              A.CLAHE(p=1.0),
              A.RandomBrightness(p=1.0),
              A.RandomGamma(p=1.0),
              A.ISONoise(p=1.0),
              A.IAASharpen(p=1.0),
              A.Blur(blur_limit=3, p=1.0),
              A.MotionBlur(blur_limit=3, p=1.0),
              A.RandomContrast(p=1.0),
              A.HueSaturationValue(p=1.0),
              A.Resize(height=height, width=width, p=1.0),
              A.Cutout(p=1.0),
              A.PadIfNeeded(pad_to_multiple(height), 
                            pad_to_multiple(width), 
                            border_mode=cv2.BORDER_CONSTANT, 
                            value=0, 
                            mask_value=0)
             ]

In [29]:
cv2.imwrite(os.path.join(target_dir, 'original.png'), img)
for name, transform in zip(transform_names, transforms):
    new = transform(image=img)['image']
    cv2.imwrite(os.path.join(target_dir, f'{name}.png'), new)